In [18]:
import os
import openai
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import AzureSearch
from dotenv import load_dotenv
load_dotenv('/Users/alfredo/.secrets/azure-open-ai.sh')

# configure openai
# openai.api_type = "azure"
# openai.api_base = os.getenv('OPENAI_API_BASE')
# openai.api_key = os.getenv('AZURE_OPENAI_API_KEY')
# openai.api_version = "2023-07-01-preview"
# print(openai.api_base)

True

In [ ]:
!env | grep AZURE

In [ ]:
embeddings = OpenAIEmbeddings(deployment="demo-embedding", chunk_size=1)

# Connect to Azure Cognitive Search
acs = AzureSearch(azure_search_endpoint=os.getenv('AZURE_COGNITIVE_SEARCH_SERVICE_NAME'),
                 azure_search_key=os.getenv('AZURE_COGNITIVE_SEARCH_API_KEY'),
                 index_name=os.getenv('AZURE_COGNITIVE_SEARCH_INDEX_NAME'),
                 embedding_function=embeddings.embed_query)

In [ ]:
from langchain.document_loaders import CSVLoader

loader = CSVLoader("wine-ratings.csv")
documents = loader.load()

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

acs.add_documents(documents=docs)

In [ ]:
docs = acs.similarity_search_with_relevance_scores(
    query="What is the best Cabernet Sauvignon wine in Napa Valley above 94 points",
    k=5,
)
print(docs[0][0].page_content)
print(dir(docs[0][0]))

In [23]:
openai.api_base = os.getenv("OPENAI_API_BASE")  # Your Azure OpenAI resource's endpoint value.
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_type = "azure"
openai.api_version = "2023-05-15" 
messages=[
    {"role": "system", "content": "Asisstant is a chatbot that helps you find the best wine for your taste."},
    {"role": "user", "content": "What is the best Cabernet Sauvignon wine in Napa Valley above 94 points?"},
    {"role": "assistant", "content": docs[0][0].page_content}
]

response = openai.ChatCompletion.create(
    engine="demo-alfredo",
    messages=messages,
)
from pprint import pprint
pprint(response)
print(response['choices'][0]['message']['content'])

/openai/deployments/demo-alfredo/chat/completions?api-version=2023-05-15
(': 36\n'
 'name: Shafer "Hillside Select" Napa Valley Cabernet Sauvignon 2015\n'
 'grape: Cabernet Sauvignon\n'
 'region: Stags Leap District, Napa Valley, California\n'
 'variety: Red Wine\n'
 'rating: 98.0\n'
 'notes: This wine offers a distinctive nose of dark chocolate, black cherry, '
 'and cassis. On the palate, it is rich and full-bodied with intense flavors '
 'of blackberry, plum, and mocha. The tannins are firm yet refined, with a '
 'long and vibrant finish. It is a highly acclaimed wine that showcases the '
 'best of Napa Valley Cabernet Sauvignon.')
